<a href="https://colab.research.google.com/github/MatveyOreshkinn/Minor/blob/main/%D0%9B%D0%A01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import re

from google.colab import drive
import warnings


warnings.filterwarnings("ignore")
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
log_pattern = r'(\S+) - - \[(.*?)\] "(\S+) (\S+) (\S+\s\S+)" (\d+) (\d+) (\S+)'
# Создаем пустой список для хранения распарсенных данных
parsed_data = []

# Открываем файл и читаем построчно
with open('/content/drive/MyDrive/access.log', 'r') as file: # Убедитесь, что указали правильный путь к файлу
    for line in file:
        # Ищем совпадения с нашим шаблоном в каждой строке
        match = re.match(log_pattern, line)
        if match:
            # Если строка соответствует шаблону, извлекаем группы
            parsed_data.append(match.groups())

# Определяем названия колонок для будущего DataFrame
column_names = ['ip', 'timestamp', 'http_method', 'endpoint', 'http_version', 'status_code', 'response_size', 'session_id']

# Создаем DataFrame
df = pd.DataFrame(parsed_data, columns=column_names)
df

,ip,timestamp,http_method,endpoint,http_version,status_code,response_size,session_id
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,HTTP 1.1,200,4407,ID4887
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,HTTP 1.1,200,4025,ID4887
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,HTTP 1.1,200,3482,ID4887
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,HTTP 1.1,200,4904,ID4887
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,HTTP 1.1,200,2104,ID4887
...,...,...,...,...,...,...,...,...
239197,121.64.17.46,31/Jan/2015:23:59:40 +03:00,POST,/catalog.phtml,HTTP 1.1,200,2760,ID55681
239198,13.190.220.64,31/Jan/2015:23:59:47 +03:00,POST,/search.phtml,HTTP 1.1,200,2778,ID55720
239199,121.64.17.46,31/Jan/2015:23:59:51 +03:00,POST,/search.phtml,HTTP 1.1,200,2881,ID55681
239200,2.57.156.197,31/Jan/2015:23:59:56 +03:00,POST,/search.phtml,HTTP 1.1,200,3288,ID55721
